In [2]:
import os
import trimesh
import random
import torch
from tqdm import tqdm
import pickle

from scripts.utils import save_mesh_with_color_clinical, save_mesh_with_color_acdc, dir_checker
from scripts import postprocessing

import numpy as np
from sklearn.utils import shuffle
from sklearn import svm
import gc

# machine learning
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC, LinearSVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.linear_model import Perceptron
from sklearn.linear_model import SGDClassifier
from sklearn.tree import DecisionTreeClassifier

from scripts.models import Segmentation_Network_full

model_dir = './models/clinical_no_TL_1'
target = 'best' # 'last'
model = torch.load(f'{model_dir}/{target}.pt').cpu()
model.eval()


Segmentation_Network_full(
  (network): Sequential(
    (0): Conv3d(1, 32, kernel_size=(2, 2, 2), stride=(2, 2, 2))
    (1): ReLU()
    (2): Conv3d(32, 64, kernel_size=(2, 2, 2), stride=(2, 2, 2))
    (3): ReLU()
    (4): Conv3d(64, 128, kernel_size=(2, 2, 2), stride=(2, 2, 2))
    (5): ReLU()
    (6): Conv3d(128, 256, kernel_size=(2, 2, 2), stride=(2, 2, 2))
    (7): ReLU()
    (8): Conv3d(256, 256, kernel_size=(1, 1, 1), stride=(1, 1, 1))
    (9): ReLU()
    (10): Conv3d(256, 256, kernel_size=(1, 1, 1), stride=(1, 1, 1))
    (11): ReLU()
    (12): Conv3d(256, 4, kernel_size=(1, 1, 1), stride=(1, 1, 1))
    (13): Sigmoid()
    (14): Flatten(start_dim=1, end_dim=-1)
  )
)

In [6]:
sdfs = []
colors = []
preprocessed_dir = 'TL_prepared_data/sdf'

for name in os.listdir(preprocessed_dir):
    sdf = torch.load(f'{preprocessed_dir}/{name}/sdf.pt')
    sdf = sdf.view(len(sdf), -1)
    
    color = torch.load(f'{preprocessed_dir}/{name}/color.pt')
    color= torch.argmax(color, dim=1)

    sdfs.append(sdf)
    colors.append(color)
    

In [ ]:
model_dir = './models/no_nn/'
for m in tqdm(os.listdir(model_dir)):
    path_out = './predictions/inference'
    dir_in = './data_for_inference/obj/'
    name = '045.obj'
    
    with open(model_dir + m, 'rb') as f:
        model = pickle.load(f)

    preprocessed_dir = 'TL_prepared_data/sdf'
    preprocessed_dir += '/' + name[:-4]
    
    mesh = trimesh.load(dir_in+name)

    sdf = torch.load(f'{preprocessed_dir}/sdf.pt')
    sdf = sdf.view(len(sdf), -1)
    
    colors = torch.load(f'{preprocessed_dir}/color.pt')
    colors= torch.argmax(colors, dim=1)

    predicted = model.predict(sdf.float())
    
    name = name[:-4] + '_' + m
    
    dirname = dir_checker(path_out, name)
    save_mesh_with_color_clinical(mesh, colors, path=f'{dirname}/{name[:-4]}_original.obj')
    save_mesh_with_color_clinical(mesh, predicted, path=f'{dirname}/{name[:-4]}_predicted.obj')
    
    mesh = trimesh.load(f'{dirname}/{name[:-4]}_predicted.obj')
    
    postprocessing(mesh, f'{dirname}/{name[:-4]}_postprocessed.obj')

